# CSS 201 / 202 - CSS Bootcamp

## Week 06 - Lecture 03

### Umberto Mignozzetti

## (If you want to) Configure DataHub

1. Open Terminal.

2. Type (substituting the **\<XXXXyour-user-hereXXXX\>** for your actual username).

```
mkdir mykernel
python3 -m venv mykernel
source mykernel/bin/activate

which pip # output = /datasets/home/.../<XXXXyour-user-hereXXXX>/mykernel/bin/pip
/home/<XXXXyour-user-hereXXXX>/mykernel/bin/python3 -m pip install --upgrade pip

pip install ipython ipykernel scikit-learn seaborn statsmodels plotly

which ipython # output = /datasets/home/.../<XXXXyour-user-hereXXXX>/mykernel/bin/pip/ipython
ipython kernel install --user --name=mykernel
deactivate
```

3. Close and open again your DataHub, now you have all up-to-date!

# Resampling

In [ ]:
## Loading the relevant packages
import pandas as pd
import numpy as np

# Plotting things:
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

# Loading scikit learn relevant packages (note our new friends!)
from sklearn.linear_model import LogisticRegression, LinearRegression, Ridge, Lasso
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.metrics import confusion_matrix, classification_report, precision_score, get_scorer_names, mean_squared_error
from sklearn.model_selection import train_test_split, LeaveOneOut, cross_val_score, KFold, GridSearchCV
from sklearn.inspection import DecisionBoundaryDisplay
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.feature_selection import SequentialFeatureSelector

## Resampling

- Involve repeatedly drawing `samples` for a `training dataset` to obtain fitting information.

- `Samples`: A randomly selected fraction of the original data.
    + Do not mistake it for a different sample from a population.
    
- `Training`: Training the model means to fit the model.

## Resampling

- This sounds weird: why not fit the model into the actual data?
    + We would not have a measure of how well our model is doing.
    + In the end, this matters! And matters especially for the data that we did not train the model!

- In this sense, resampling is a clever trick to see how the model would do in the `real world`, without going to the real world.

## Resampling

- It helps us to:
    + Evaluate the performance of the model (`Model assessment`).
    + Select the proper flexibility for our model (`Model selection`).

- Drawback: they are computationally intensive.
    + Usually involves refitting the model again and again.
    
- We are going to discuss the following:
    + `Cross-validation`: Measure the performance and select appropriate flexibility.
    + (not this now) `Bootstrap`: Measure the accuracy of parameters.

## Resampling

In [ ]:
## Loading Chile data
chile = pd.read_csv('https://raw.githubusercontent.com/umbertomig/POLI175public/main/data/chilesurvey.csv')
chile_clean = chile.dropna()
chile_clean = chile_clean[chile_clean['vote'].isin(['Y', 'N'])]
chile_clean['vote'] = np.where(chile_clean['vote'] == 'Y', 1, 0)
chile_clean['logincome'] = np.log(chile_clean['income'])
chile_clean['logpop'] = np.log(chile_clean['population'])
dummies = pd.get_dummies(chile_clean['sex'], prefix = 'sex', drop_first = True)
chile_clean = pd.concat([chile_clean, dummies], axis=1)
dummies = pd.get_dummies(chile_clean['region'], prefix = 'region', drop_first = True)
chile_clean = pd.concat([chile_clean, dummies], axis=1)
dummies = pd.get_dummies(chile_clean['education'], prefix = 'education', drop_first = True)
chile_clean = pd.concat([chile_clean, dummies], axis=1)
chile_clean.head()

## Resampling

In [ ]:
## Education Expenditure Dataset
educ = pd.read_csv('https://raw.githubusercontent.com/umbertomig/POLI175public/main/data/educexp.csv')
educ = educ.set_index('states')
for i in educ.columns:
    educ[i + '_log'] = np.log(educ[i])
educ.head()

## Cross-Validation

- We talked about it yesterday.

- In that context, we looked at the idea of a
    + `training error rate` (the boring one): The error when fitting the model to data that was used to train the parameters, and
    + `test error rate` (the cool one): The error associated with fitting the model to ***unseen*** data.

## Cross-Validation

### Validation Set Approach

- Randomly divide the data into two sets:
    + `Training set`: The data used to fit the model
    + `Testing set`: The data used to test the performance of the fitted model.

## Cross-Validation

### Validation Set Approach

- Split the sample in half training - half testing and running the estimation:

![img vsa](https://github.com/umbertomig/POLI175public/blob/main/img/cv1.png?raw=true)

## Cross-Validation

### Validation Set Approach

In [ ]:
## With 50% split (no urban_log)
y = educ['education_log']
X = educ[['income_log', 'young_log']]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.5, random_state = 1234)

reg = LinearRegression().fit(X_train, y_train)

y_pred = reg.predict(X_test)

np.sum((y_pred - y_test) ** 2)

## Cross-Validation

### Validation Set Approach

In [ ]:
## With 50% split (with urban_log)
y = educ['education_log']
X = educ[['income_log', 'young_log', 'urban_log']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1, random_state = 1234)
reg = LinearRegression().fit(X_train, y_train)
y_pred = reg.predict(X_test)
np.sum((y_pred - y_test) ** 2)

## Cross-Validation

### Validation Set Approach

In [ ]:
## Your turn: Check the MSE when removing income_log. Is it
##  better?

## Cross-Validation

### Validation Set Approach

In [ ]:
## Your turn: Check the MSE when removing 'urban_pop' 
##   with only 20% of observations in the testing set.

## Cross-Validation

### Leave-One-Out Cross-Validation

- It does what it says: leaves one observation out and fits the model with $n-1$ cases.

- Then, it predicts the results in the case left out.

- **Great** for small datasets and when prediction is critical.

- **Bad** in terms of computational time.

$$ CV_n \ = \ \dfrac{1}{n}\sum_i MSE_i $$

## Cross-Validation

### Leave-One-Out Cross-Validation

![img](https://github.com/umbertomig/POLI175public/blob/main/img/cv2.png?raw=true)

## Cross-Validation

### Leave-One-Out Cross-Validation

In [ ]:
## LOOCV
## Variables: model without urban population
y = educ['education_log']
X = educ[['income_log','young_log']]

## Leave-One-Out-CV
cv = LeaveOneOut()
reg = LinearRegression()

## Run the CV
scores = cross_val_score(reg, X, y,
                         scoring = 'neg_mean_squared_error',
                         cv = cv)

## RMSE
print(np.sqrt(np.mean(np.absolute(scores))))

## MSE
np.mean(np.absolute(scores))

## Cross-Validation

### Leave-One-Out Cross-Validation

In [ ]:
## LOOCV
## Variables: model **with** urban population
y = educ['education_log']
X = educ[['income_log', 'young_log', 'urban_log']]

## Leave-One-Out-CV
cv = LeaveOneOut()
reg = LinearRegression()


## Run the CV
scores = cross_val_score(reg, X, y, 
                         scoring = 'neg_mean_squared_error',
                         cv = cv)

## MSE
print(np.mean(np.absolute(scores)))

## RMSE
np.sqrt(np.mean(np.absolute(scores)))

## Cross-Validation

### Leave-One-Out Cross-Validation

In [ ]:
## Your turn: compare the model with x without logs
## Note: the target has to be the same!

## Cross-Validation

### Metrics

- To do the comparison, you need a metric.

- `scikit learn` has many matrics available:

In [ ]:
## Lots of stats to compute the error:
print(get_scorer_names())

## Cross-Validation

### Leave-One-Out Cross-Validation

In [ ]:
## Your turn: find and use R-squared as the parameter for a
## LOOCV. What is the difference?

## Cross-Validation

### K-Fold Cross-Validation

- Leaves $k$ groups out and fits the model with the observations outside each group.

- Then, it predicts the results in the cases left out.

- **Great** in most cases.

- **Bad** *sometimes* computationally expensive.

$$ CV_k \ = \ \dfrac{1}{k}\sum_i MSE_i $$

## Cross-Validation

### K-Fold Cross-Validation

![img](https://github.com/umbertomig/POLI175public/blob/main/img/cv3.png?raw=true)

## Cross-Validation

### K-Fold Cross-Validation

In [ ]:
## K-Fold CV (k = 5)
y = educ['education_log']
X = educ[['income_log', 'young_log']]

## k-Fold CV (n_splits = k, shuffle: reshuffle data before split)
cv = KFold(n_splits = 5, random_state = 1234, shuffle = True) 
reg = LinearRegression()


## Run the CV
scores = cross_val_score(reg, X, y,
                         scoring = 'neg_mean_squared_error',
                         cv = cv)

## MSE
print(np.mean(np.absolute(scores)))

## RMSE
np.sqrt(np.mean(np.absolute(scores)))

## Cross-Validation

### K-Fold Cross-Validation

In [ ]:
## K-Fold CV (k = 5)
y = educ['education_log']
X = educ[['income_log', 'young_log', 'urban_log']]

## k-Fold CV (n_splits = k, shuffle: reshuffle data before split)
cv = KFold(n_splits = 5, random_state = 1234, shuffle = True) 
reg = LinearRegression()


## Run the CV
scores = cross_val_score(reg, X, y,
                         scoring = 'neg_mean_squared_error',
                         cv = cv)

## MSE
print(np.mean(np.absolute(scores)))

## RMSE
np.sqrt(np.mean(np.absolute(scores)))

## Cross-Validation

### K-Fold Cross-Validation

In [ ]:
## Your turn: Run a 10-fold CV? Any differences?

## Cross-Validation

### Bias-Variance Trade-off

- k-Fold CV is more computationally efficient than LOOCV. But how about Bias-Variance Trade-offs?

- Larger fractions in a two-split leads to high bias: over-estimates the error rates.

- LOOCV: leaves just one, so it gives an unbiased estimate of the testing error rates: 
    + Very good for bias reduction!

## Cross-Validation

### Bias-Variance Trade-off

- LOOCV has high variance: almost the same observations at each run!
    + Very bad for variance.
    
- k-Fold CV:
    + Each subset is a *bit more different* than the other.
    + Leads to less correlation between each fold.
    + Good balance usually with $k=5$ or $k=10$.

## Cross-Validation

### Bias-Variance Trade-off

![img](https://github.com/umbertomig/POLI175public/blob/main/img/cv4.png?raw=true)

## Cross-Validation

### CV on Classification Problems

- When we have a classification, we must change how we evaluate the error.

- With classification, the LOOCV would look like this:

$$ CV_n \ = \ \dfrac{1}{n} \sum_i I(y_i \neq \widehat{y}_i) $$

- And the `accuracy` measure will be $I(y_i = \widehat{y}_i)$, so we need to subtract 1.

## Cross-Validation

### CV on Classification Problems

![img](https://github.com/umbertomig/POLI175public/blob/main/img/cv5.png?raw=true)

## Cross-Validation

### CV on Classification Problems

In [ ]:
## LOOCV on a Logistic Regression
# Checking best polynomial for Age
poly = list(range(1, 6))
errmea = []
y = chile_clean['vote']
for p in poly:
    if p == 1:
        X = pd.DataFrame({
            'age_1': chile_clean['age']
        })
    else:
        X['age_' + str(p)] = X['age_1'] ** p
    cv = LeaveOneOut()
    logreg = LogisticRegression()
    scores = cross_val_score(logreg, X, y, 
                             scoring = 'accuracy',
                             cv = cv, n_jobs = -1)
    print('For polynomial order {a}, the Logistic Regression Error Rate is {b}.\n'.format(a = str(p), b = str(1-scores.mean())))
    errmea.append(1-scores.mean())

## Classification

### K-Nearest Neighbors Classifier

- Little detour back to talk about a good algorithm for classification (also very intuitive).

- Given an integer $K$, and a test observation, it says that:

$$ \mathbb{P}(Y = j| X = x_0) \ = \ \dfrac{1}{K}\sum_{i \in N_0} I(y_i = j) $$

- Meaning: classify the observation based on the class of the closest $K$ obs:
    + The one more frequent is the winner.
    
- Closest: the idea of a metric.

## Classification

### K-Nearest Neighbors Classifier

![img](https://github.com/umbertomig/POLI175public/blob/main/img/knn1.png?raw=true)

## Classification

### K-Nearest Neighbors Classifier

![img](https://github.com/umbertomig/POLI175public/blob/main/img/knn2.png?raw=true)

## Classification

### K-Nearest Neighbors Classifier

In [ ]:
# KNN
X = chile_clean[['age', 'statusquo']]
y = chile_clean['vote']

# Create the model
knn = KNeighborsClassifier(n_neighbors = 10).fit(X, y)

# Plotting the tree boundaries
fig = DecisionBoundaryDisplay.from_estimator(knn, X, response_method="predict",
                                             alpha=0.5, cmap=plt.cm.coolwarm)

# Plotting the data points    
fig.ax_.scatter(x = chile_clean['age'], y = chile_clean['statusquo'], 
                c = y, alpha = 0.5,
                cmap = plt.cm.coolwarm)

plt.show()

## Classification

### K-Nearest Neighbors Classifier

In [ ]:
## Now choose K! (We will learn a better method for doing this: GridSearchCV!)
bigK = list(range(1, 100))
errmea = []
y = chile_clean['vote']
X = chile_clean[['statusquo', 'logincome', 'logpop', 'age']]
for smallk in bigK:
    cv = KFold(n_splits = 10, random_state = 1234, shuffle = True)
    knn = KNeighborsClassifier(n_neighbors = smallk)
    scores = cross_val_score(knn, X, y, 
                             scoring = 'accuracy',
                             cv = cv, n_jobs = -1)
    errmea.append(1-scores.mean())
print('Best K is {a}.'.format(a = str(bigK[errmea.index(min(errmea))])))

## Classification

### K-Nearest Neighbors Classifier

In [ ]:
sns.lineplot(x = bigK, y = errmea)
plt.title('KNN algorithm')
plt.xlabel('K')
plt.ylabel('Error Rate')
plt.scatter(bigK[errmea.index(min(errmea))], min(errmea), marker='X', color = 'red')
plt.show()

## Classification

**Check-in:** Study the best method for predicting default in credit card.

In [ ]:
default = pd.read_csv('https://raw.githubusercontent.com/umbertomig/POLI175public/main/data/default.csv')
default.head()

# Linear Model Selection and Regularization

## Linear Model Selection and Regularization

- We usually have a large set of predictors that could be used.
    + Which predictors to pick becomes a task.

- If we are trying to interpret things and learn from the data, then which predictors are correlated with the outcome is informative:
    + Again, picking predictors becomes a task.
    
- We will learn how to do that systematically.
    - We are going to consider techniques to select a subset of predictors based on a performance metric.
    - Later we will see other techniques

## Linear Model Selection and Regularization

### Subset Selection

#### Best Subset Selection

**Algorithm:**

1. Let $M_0$ denote the null model, which contains no predictors. This model predicts the sample mean for each observation.

2. For $k = \{1, 2, \cdots, p\}$:
    1. Fit all $p \choose k$ models with exactly $k$ predictors.
    2. Pick the *best* among these models and call it your $M_k$

3. Select a single best model from among $M_0, \cdots, M_p$ using cross-validated prediction error, $C_p$, AIC, BIC, or adjusted $R^2$.

## Linear Model Selection and Regularization

### Subset Selection

#### Best Subset Selection

![img ms1](https://github.com/umbertomig/POLI175public/blob/main/img/ms1.png?raw=true)

## Linear Model Selection and Regularization

### Subset Selection

#### Best Subset Selection

Notes:

1. You can do this with Logistic Regression: change RSS with [*deviance*](https://en.wikipedia.org/wiki/Deviance_(statistics)).
    + In this case, our friend $- 2\ln({\hat {L}})$ does very well!

2. Best Selection is excellent but fits around $2^p$ models.
    + $p = 10$ means around 1000 estimates.

## Linear Model Selection and Regularization

### Subset Selection

#### Stepwise Selection: Forward Selection

**Algorithm**:

1. Let $M_0$ denote the null model, which contains no predictors.

2. For $k = \{0, 1, 2, \cdots, p-1\}$:
    1. Consider all $p - k$ models that augments $M_k$ by one predictor.
    2. Pick the *best* among these $p-k$ models, and call it your $M_{k+1}$.

3. Select a single best model from among $M_0, \cdots, M_p$ using cross-validated prediction error, $C_p$, AIC, BIC, or adjusted $R^2$.

## Linear Model Selection and Regularization

### Subset Selection

#### Stepwise Selection: Forward Selection

- Much more efficient:
    + It fits a total of $1 + \dfrac{p(p+1)}{2}$ models.
    + If $p = 20$, the Best Selection would fit 1,048,576
    + If $p = 20$, the Forward Step Selection would fit 211 models.

## Linear Model Selection and Regularization

### Subset Selection

#### Stepwise Selection: Forward Selection

- The catch: It is not guaranteed that it is going to find the *best subset* model.

- Example: Let $p = 3$.
    + Suppose that the best model involves $v2$ and $v3$.
    + But suppose that within the models with only one variable, $v1$ would do better.
    + Then, Forward Step Selection would never pick this model!

## Linear Model Selection and Regularization

### Subset Selection

#### Stepwise Selection: Forward Selection

![img ms2](https://github.com/umbertomig/POLI175public/blob/main/img/ms2.png?raw=true)

## Linear Model Selection and Regularization

### Subset Selection

#### Stepwise Selection: Backward Selection

**Algorithm**:

1. Let $M_p$ denote the *full* model, which contains all $p$ predictors.

2. For $k = \{p, p-1, p-2, \cdots, 1\}$:
    1. Consider all $k$ models that contail all but one predictor in $M_k$, for a total of $k-1$ predictors.
    2. Pick the *best* among these $k$ models, and call it your $M_{k-1}$.

3. Select a single best model from among $M_0, \cdots, M_p$ using cross-validated prediction error, $C_p$, AIC, BIC, or adjusted $R^2$.

## Linear Model Selection and Regularization

### Subset Selection

#### Stepwise Selection: Backward Selection

- Computational Efficiency:
    + It fits a total of $1 + \dfrac{p(p+1)}{2}$ models.
    + Same efficiency as the Forward Selection.

- Catch:
    + Same catch as the Forward Selection: It does not guarantee the pick of the best model.

## Linear Model Selection and Regularization

### Subset Selection

#### Stepwise Selection: Hybrid Approaches

- Combinations of *Forward* and *Backwards* that intend to mimic the *Best Selection*.

- Many available.

- But the trade-offs are clear: 
    + Computational efficiency
    + Likelihood of picking the best model

## Linear Model Selection and Regularization

### Subset Selection

#### What is Best?

- For each of the $M_k$ models, this is it:
    + RSS: Residual Sum of Squares: We want it to be the lowest possible.
    + $R^2$: We want it to be the highest possible.
    + And for Logistic or other GLM Regressions, *deviance*.

## Linear Model Selection and Regularization

### Subset Selection

#### What is Best?

- Catches: 
    1. RSS and $R^2$ always improve with more variables.
    2. We want to look at the *testing set goodness-of-fit*, not the *training sets goodness-of-fit*!

- And that is why RSS and $R^2$ are not used in Step 3:
    - We need something that eventually gets worse the more variables we throw in.

## Linear Model Selection and Regularization

### Subset Selection

#### What is Best?

**Important:**

- Training Set MSE generally underestimates the testing set MSE.

$$ \text{MSE} \ = \ \dfrac{RSS}{n} $$

- But before, we could not split data into *training* and *testing*. 
    + This is a more recent feature, thanks to our increased computational power.

- Here are a few stats that we can fit in the training set.

## Linear Model Selection and Regularization

### Subset Selection

#### What is Best?

- [$C_P$](https://en.wikipedia.org/wiki/Mallows%27s_Cp) in a model containing:
    - $d$ predictors.
    - $n$ observations.
    - $\widehat{\sigma}^2$ the variance of the error in the full model with all predictors.

$$ C_p \ = \ \dfrac{1}{n}(RSS + 2\times d \times \widehat{\sigma}^2) $$

- The smaller, the better.

## Linear Model Selection and Regularization

### Subset Selection

#### What is Best?

- [AIC](https://en.wikipedia.org/wiki/Akaike_information_criterion) or $C_p$: AIC is a goodness-of-fit parameters that goes lower when you are improving the model
    + But for every variable you add, it penalizes it.
    + If by adding more variables, it goes up, then your model is getting more complex without adding much.
    
$$ \text{AIC} \ = \ 2d - 2\log({\hat {L}}) $$

## Linear Model Selection and Regularization

### Subset Selection

#### What is Best?

- The maximum likelihood and least squares are the same for models with Gaussian errors.

$$ \text{AIC} \ = \ \dfrac{1}{n} (RSS + 2\times d \times \widehat{\sigma}^2) $$

## Linear Model Selection and Regularization

### Subset Selection

#### What is Best?

- [BIC](https://en.wikipedia.org/wiki/Bayesian_information_criterion) is another goodness-of-fit, but this one penalizes the addition of variables at a higher rate.

$$ \text{BIC} = k\log(n) - 2\log({\widehat {L}}) $$

- Note the difference from the AIC: instead of multiplying by 2, it is multiplying by $\ln(n)$!

- Again, lower values are better.

## Linear Model Selection and Regularization

### Subset Selection

#### What is Best?

- Or if you want the one with the least square errors:

$$ \text{AIC} \ = \ \dfrac{1}{n} (RSS + \log(n) \times d \times \widehat{\sigma}^2)  $$ 

## Linear Model Selection and Regularization

### Subset Selection

#### What is Best?

- [Adjusted $R^2$](https://en.wikipedia.org/wiki/Bayesian_information_criterion): It is a change in the $R^2$ to penalize the addition of regressors.

$$ \overline{R}^{2} = 1-(1-R^{2})\dfrac{n-1}{n-d} \ = \ 1 - \dfrac{\frac{RSS}{n - d - 1}}{\frac{TSS}{n - 1}}$$

- $R^2$ always increase but the $\overline{R}^2$ may increase or decrese.

- **Not like the others:** This one, the higher, the better.

## Linear Model Selection and Regularization

### Subset Selection

#### What is Best?

![img ms3](https://github.com/umbertomig/POLI175public/blob/main/img/ms3.png?raw=true)

## Linear Model Selection and Regularization

### Subset Selection

#### What is Best?

- $C_P$, AIC, and BIC all have a solid theoretical justification.
    + Many of them have something we call Large Sample Properties.
    + Check their Wikipedia of them. They converge to nice, important values.

- $\overline{R}^{2}$ does not.

## Linear Model Selection and Regularization

### Subset Selection

#### What is best? Validation and Cross-Validation

- The main advantage is obvious: You look at testing errors!

- The other main advantage is regarding estimating parameters:
    + $C_P$, AIC, and BIC all have a strong theoretical justification, which is reassuring.
    + But sometimes, one does not know the theory behind an estimate to compute statistics or even standard errors.
    + E.g., which $\widehat{\sigma}^2$ should we pick?
    + Validation and Cross-Validation do great in these cases!

## Linear Model Selection and Regularization

### Subset Selection

#### What is best? Validation and Cross-Validation

![img ms4](https://github.com/umbertomig/POLI175public/blob/main/img/ms4.png?raw=true)

## Linear Model Selection and Regularization

### Subset Selection

In [ ]:
## Loading Chile data
chile = pd.read_csv('https://raw.githubusercontent.com/umbertomig/POLI175public/main/data/chilesurvey.csv')
chile_clean = chile.dropna()
chile_clean = chile_clean[chile_clean['vote'].isin(['Y', 'N'])]
chile_clean['vote'] = np.where(chile_clean['vote'] == 'Y', 1, 0)
chile_clean['logincome'] = np.log(chile_clean['income'])
chile_clean['logpop'] = np.log(chile_clean['population'])
dummies = pd.get_dummies(chile_clean['sex'], prefix = 'sex', drop_first = True)
chile_clean = pd.concat([chile_clean, dummies], axis=1)
dummies = pd.get_dummies(chile_clean['region'], prefix = 'region', drop_first = True)
chile_clean = pd.concat([chile_clean, dummies], axis=1)
dummies = pd.get_dummies(chile_clean['education'], prefix = 'education', drop_first = True)
chile_clean = pd.concat([chile_clean, dummies], axis=1)
chile_clean.head()

## Linear Model Selection and Regularization

### Subset Selection

In [ ]:
## Education Expenditure Dataset
educ = pd.read_csv('https://raw.githubusercontent.com/umbertomig/POLI175public/main/data/educexp.csv')
educ = educ.set_index('states')
for i in educ.columns:
    educ[i + '_log'] = np.log(educ[i])
educ.head()

## Stepwise Subset Selection: Code

In [ ]:
## Let us predict the education expenditure using
## raws, squares, and cubic powers of the predictors
raws = ['income', 'young', 'urban']
y = educ['education']
X = educ.reset_index()[raws]
for power in range(2, 4):
    for var in raws:
        X[var + '_' + str(power)] = X[[var]] ** power

# Found on stackoverflow
def powerSet(s):
    sets = [[]]
    for i in s:
        newsets = []
        for k in sets:
            newsets.append(k+[i])
        sets += newsets
    return sets

## Stepwise Subset Selection: Code

In [ ]:
## My best subset selection code
def best_subset_selection(est, X, y, cvn = 5):
    modsAll = powerSet(X.columns)
    mods = {}
    for i in modsAll:
        if i == []:
            mods[0] = {'model': 'Null',
                       'RSS': np.sum((y - np.mean(y)) ** 2)}
        else:
            rss = np.sum((y - est.fit(X[i], y).predict(X[i])) ** 2)
            if len(i) not in mods:
                mods[len(i)] = {'model': i, 'RSS': rss}
            else:
                if mods[len(i)]['RSS'] > rss:
                    mods[len(i)] = {'model': i, 'RSS': rss}
    bestmod = mods.pop(0)
    bestmod.pop('RSS')
    bestmod['Rsquared'] = 0
    for i in mods.values():
        score = cross_val_score(est, X[i['model']], y, cv = cvn, scoring = 'r2').mean()
        if bestmod['Rsquared'] < score:
            bestmod['model'] = i['model']
            bestmod['Rsquared'] = score
    return(bestmod)

print(best_subset_selection(LinearRegression(), X, y))

## Forward Subset Selection: Code

In [ ]:
## My forward subset selection
def forward_subset_selection(est, X, y, cvn = 5):
    variables = list(X.columns)
    mods = {}
    mods[0] = {'model': [], 'RSS': np.sum((y - np.mean(y)) ** 2)}
    for i in range(1, len(variables) + 1):
        mods[i] = {'model': mods[i-1]['model'], 'RSS': mods[i-1]['RSS']}
        for j in [item for item in variables if item not in mods[i-1]['model']]:
            varsaux = mods[i-1]['model'] + [j]
            rss = np.sum((y - est.fit(X[varsaux], y).predict(X[varsaux])) ** 2)
            if mods[i]['RSS'] > rss:
                mods[i]['model'] = varsaux
                mods[i]['RSS'] = rss
    bestmod = mods.pop(0)
    bestmod.pop('RSS')
    bestmod['Rsquared'] = 0
    for i in mods.values():
        score = cross_val_score(est, X[i['model']], y, cv = cvn, scoring = 'r2').mean()
        if bestmod['Rsquared'] < score:
            bestmod['model'] = i['model']
            bestmod['Rsquared'] = score
    return(bestmod)

print(forward_subset_selection(LinearRegression(), X, y))

## Forward Subset Selection: Code

In [ ]:
## Forward Stepwise Selection in Scikit Learn:
reg = LinearRegression()
sfs = SequentialFeatureSelector(reg, n_features_to_select='auto', direction = 'forward')
sfs.fit(X, y)
X.columns[sfs.get_support()]

## Backward Subset Selection: Code

In [ ]:
## Backward Stepwise Selection in Scikit Learn:
reg = LinearRegression()
sfs = SequentialFeatureSelector(reg, n_features_to_select='auto', direction = 'backward')
sfs.fit(X, y)
X.columns[sfs.get_support()]

## Backward Subset Selection: Code

In [ ]:
## Backward Stepwise Selection in Scikit Learn. Limiting to tops three features.
reg = LinearRegression()
sfs = SequentialFeatureSelector(reg, n_features_to_select=3, direction = 'backward')
sfs.fit(X, y)
X.columns[sfs.get_support()]

## Linear Model Selection and Regularization

- Now we are going to focus on two of the most used methods for model selection:
    + **Ridge**
    + **Lasso**

- These methods are different from the subset selection in special ways:
    + They are meant to "*shrink*" the coefficients toward zero!

- It may be counter-intuitive, but these methods are great tools to reduce the variance of the estimates (recall the bias-variance trade-offs).

## Ridge Regression

- Instead of minimizing the RSS, one minimizes:

$$ RSS + \underbrace{\alpha \sum_{j=1}^p\beta_j^2}_{\text{shrinkage penalty}} $$

- The $\alpha \geq 0$ parameters is called *tuning parameter*. 

- Selecting a good $\alpha$ is crucial for a good set of estimates.

In [ ]:
## Ridge Regression (badly done)
ridge = Ridge(alpha = 1).fit(X, y)
print('The R-squared for this regression is: ' + str(ridge.score(X, y)))
plt.bar(X.columns, ridge.coef_)
plt.xticks(rotation=45)
plt.show()

## Ridge Regression

- $\alpha = 0$ is the same as the least square regression.

- As $\alpha$ grows, the shrinkage penalty increases, and the ridge coefficients approach zero.

**Caveat 1**: The scale of the variable influences the results.

- In a OLS (standard least square regression), when you multiply $x_j$ by $c \neq 0$, you divide $\beta_j$ by $\dfrac{1}{c}$.

- Example: If you measure GDP in USD vs Millions of USD changes the ridge regression coefficient significantly.

**Suggestion**: Standardize the variables before running the regression

## Ridge Regression

### Standardization

- Let a variable $x_j$. Then, the standardized variable $z_j$ is obtained by:
    1. Subtracting the mean of the variable $x_j$ ($\overline{x}_j$) and then,
    2. Dividing the result by the standard deviation ($\sigma_{x_j}$) of the variable $x_j$.

$$ z_j \ = \ \dfrac{x_j - \overline{x}_j}{\sigma_{x_j}} $$

- The resulting variable $z_j$ has mean zero, variance one, and has no unit!

- Variations of one unit are called *deviations*: In a regression with a standardized variable, we say that $\beta_j$ would represent a variation on average $y$ when we increase $z_j$ by one standard-deviation.

- This is a great practice for prediction, but in general, be mindful about the unit of your data.

- **Again:** In a standard least square regression, when you multiply $x_j$ by $c \neq 0$, you divide $\beta_j$ by $\dfrac{1}{c}$.

In [ ]:
## Standardizing the X variables
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
# First observation:
print(list(X.iloc[0]))
print(list(X_scaled[0])) 

## Ridge Regression

In [ ]:
## Ridge Regression (greatly done)
ridge = Ridge(alpha = 1).fit(X_scaled, y)
print('The R-squared for this regression is: ' + str(ridge.score(X_scaled, y)))
plt.bar(X.columns, ridge.coef_)
plt.xticks(rotation=45)
plt.show()

## Ridge Regression

- **Caveat 2**: The ridge regression produces a different set of parameter for each $\alpha$.
    + $\widehat{\beta}_{\alpha}^{R}$

- Also note that the intercept ($\beta_0$) is not considered.
    + We do not want to shrink the mean of $y_i$ when $x_{ij} = 0$ for all $j$.
    + And if we standardize the variables, then the intercept will be $\widehat{\beta}_0 = \overline{y}$.

## Ridge Regression (book calls the reg parameter $\lambda$)

![img](https://github.com/umbertomig/POLI175public/blob/main/img/ridge1.png?raw=true)

## Ridge Regression

In [ ]:
## Example by Kornel Kielczewski in the sklearn documentation, adapted by me.
ridge = Ridge()
coefs = list()
errors = list()
alphas = np.logspace(-6, 6, 200)

for a in alphas:
    ridge.set_params(alpha = a).fit(X_scaled, y)
    coefs.append(ridge.coef_)
    errors.append(np.mean((ridge.predict(X_scaled) - y) ** 2))

coefs = pd.DataFrame(coefs, columns = X.columns, index = alphas)
print(errors[0:5])
coefs.head()

## Ridge Regression

In [ ]:
# Coefficients as a function of the regularization paramenter alpha
g = sns.lineplot(data = coefs)
g.set(xscale='log')
plt.show()

## Ridge Regression

In [ ]:
# Mean Squared Error as a function of the regularization paramenter alpha
g = sns.lineplot(x = alphas, y = errors)
g.set(xscale='log')
plt.show()

## Ridge Regression

- Advantage: the *non-obvious* advantage is that as $\alpha$ increses, the flexibility of the model decreases.

- This decreases variance (but increase bias).

- This may be "optimized": You may find the optimal bias-variance trade-off by manipulating $\alpha$.

## Ridge Regression

- Specially important when $p$ is close to $n$ (number of predictors is close to the number of cases).
    + This is called *high dimensional data*.

- If $p > n$, then ridge does very well. This case would have a very high variance.

- And it is *way* better than *best subset selection*: you fit just one model:
    + In practice, as many as the different $\alpha$s.
    + There are algorithms to solve efficiently for all $\alpha$s, which means that it may be more efficient than best, forward, and backward stepwise selection.

## Ridge Regression (the book calls the regularization parameter $\lambda$) 

![img](https://github.com/umbertomig/POLI175public/blob/main/img/ridge2.png?raw=true)


# Cross-Validation

## Cross-Validation

- To select the tuning parameters you can use cross-validation.

- The idea is to search through a grid of tuning parameter candidates, selecting the one that does best in the cross-validation.

- It is indeed a very straight-forward idea, if you think about it.

## Cross-Validation (the book calls the regularization parameter $\lambda$) 

![img](https://github.com/umbertomig/POLI175public/blob/main/img/cvridge.png?raw=true)

## Cross-Validation (the book calls the regularization parameter $\lambda$) 

In [ ]:
## Ridge with Cross-Validation
ridge = Ridge()
coefs = list()
errors = list()
CVerrors = list()
alphas = np.logspace(-6, 6, 200)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size = 0.25, random_state = 12345)

for a in alphas:
    ridge.set_params(alpha = a).fit(X_train, y_train)
    CVerrors.append(np.mean((ridge.predict(X_test) - y_test) ** 2))
    errors.append(np.mean((ridge.predict(X_train) - y_train) ** 2))

print('The alpha that minimizes the ridge testing set MSE is: ' + str(alphas[CVerrors.index(min(CVerrors))]))
    
mses = pd.DataFrame({
    'trainMSE': errors,
    'testMSE': CVerrors}, index = alphas)

## Cross-Validation (the book calls the regularization parameter $\lambda$) 

In [ ]:
# Cross Validation Ridge MSE as a function of the regularization parameter alpha
g = sns.lineplot(data = mses)
g.set(xscale='log')
plt.show()

# Lasso Regression

## Lasso Regression

- Ridge regression has one disadvantage: it most of the time includes $p$ predictors.

- The shrinkage never sets coefficients to be exactly zero (that is, be removed from the prediction).

- This could potentially make subset selection better than ridge.

## Lasso Regression

- But one alternative, using the same principles as the ridge regression is the **lasso** regression.

- In the lasso regression, the objective function becomes:

$$ RSS + \alpha \sum_{j=1}^p|\beta_j| $$

- Does the same as ridge: the larger the $\alpha$, the more the *shrinkage*.

## Lasso Regression

- Unlike ridge, for some values of $\alpha$, **lasso** actually force coefficients to be exactly equal to zero.

- Thus, **lasso** performs variable selection, much like the subset selection models we have seen.

- **Side-effect**: Makes models easier to interpret!
    + Yields *sparse* models: models that only involve a subset of the variables.
    
- Like ridge, selecting a good $\alpha$ is critical.

In [ ]:
## Lasso Regression (badly done)
lasso = Lasso(alpha = 1).fit(X, y)
print('The R-squared for this regression is: ' + str(lasso.score(X, y)))
plt.bar(X.columns, lasso.coef_)
plt.xticks(rotation=45)
plt.show()

In [ ]:
## Lasso Regression (greatly done)
lasso = Lasso(alpha = 1).fit(X_scaled, y)
print('The R-squared for this regression is: ' + str(lasso.score(X_scaled, y)))
plt.bar(X.columns, lasso.coef_)
plt.xticks(rotation=45)
plt.show()
print(lasso.intercept_)
print(np.mean(y))

## Lasso (the book calls the regularization parameter $\lambda$) 

![img](https://github.com/umbertomig/POLI175public/blob/main/img/lasso1.png?raw=true)

In [ ]:
## Example by Kornel Kielczewski in the sklearn Ridge documentation, adapted by me.
lasso = Lasso(max_iter = 20000000)
coefs = list()
errors = list()
alphas = np.logspace(-6, 6, 200)

for a in alphas:
    lasso.set_params(alpha = a).fit(X_scaled, y)
    coefs.append(lasso.coef_)
    errors.append(np.mean((lasso.predict(X_scaled) - y) ** 2))

coefs = pd.DataFrame(coefs, columns = X.columns, index = alphas)
print(errors[0:5])
coefs.head()

In [ ]:
# Lasso coefficients as a function of the regularization paramenter alpha
g = sns.lineplot(data = coefs)
g.set(xscale='log')
plt.show()

In [ ]:
# Lasso Mean Squared Error as a function of the regularization paramenter alpha
g = sns.lineplot(x = alphas, y = errors)
g.set(xscale='log')
plt.show()

## Lasso x Ridge Regression (the book calls the regularization parameter $\lambda$) 

- Selection property of lasso: 
    + Lasso and ridge are equivalent to a constraint on the shape of the acceptable parameter space.
    + But the "diamond shape" of lasso makes it shrinks some coefficients towards zero.

![img](https://github.com/umbertomig/POLI175public/blob/main/img/lassovsridge3.png?raw=true)

## Lasso x Ridge Regression (the book calls the regularization parameter $\lambda$) 

- Lasso performs similarly to ridge in most cases. In these cases, I'd say that lasso is better:
    + Reduces the complexity in the model.

![img](https://github.com/umbertomig/POLI175public/blob/main/img/lassovsridge1.png?raw=true)

## Lasso x Ridge Regression (the book calls the regularization parameter $\lambda$) 

- But when all coefficients are different from zero, then ridge is better.

![img](https://github.com/umbertomig/POLI175public/blob/main/img/lassovsridge2.png?raw=true)

## Lasso x Ridge Regression

- To summarize, none is better in all situations.

- You may need to search which model is better.

- Moreover, finding $\alpha$ is also a big deal. Cross-validation can help us with that!


# Cross-Validation

## Cross-Validation

- To select the tuning parameters you can use cross-validation.

- The idea is to search through a grid of tuning parameter candidates, selecting the one that does best in the cross-validation.

- It is indeed a very straight-forward idea, if you think about it.

In [ ]:
## Lasso with Cross-Validation
lasso = Lasso(max_iter = 20000000)
coefs = list()
errors = list()
CVerrors = list()
alphas = np.logspace(-6, 6, 50)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size = 0.2, random_state = 12345)

for a in alphas:
    lasso.set_params(alpha = a).fit(X_train, y_train)
    CVerrors.append(np.mean((lasso.predict(X_test) - y_test) ** 2))
    errors.append(np.mean((lasso.predict(X_train) - y_train) ** 2))

print('The alpha that minimizes the Lasso Cross-Validation MSE is: ' + str(alphas[CVerrors.index(min(CVerrors))]))
    
mses = pd.DataFrame({
    'trainMSE': errors,
    'testMSE': CVerrors}, index = alphas)

In [ ]:
# Cross Validation Lasso MSE as a function of the regularization parameter alpha
g = sns.lineplot(data = mses)
g.set(xscale='log')
plt.show()

# Beyond Linearity

## Beyond Linearity

- So far, we have focused on linear models.

- Most of the time, linear approximations are excellent:
    + Easy to interpret
    + Easy to run

- And all the GLM flavors afford lots of flexibility regarding how we deal with data.

- We will now relax the linearity assumption but keep it as simple as possible.

## Polynomial Regression

- Expands the default model ($y_i = \beta_0 + \beta_1x_i + \varepsilon_i$) to consider a polynomial $d$:

$$ y_i = \beta_0 + \beta_1x_i + \beta_2x_i^2 +\cdots + \beta_dx_i^d + \varepsilon_i $$
                     
- Lots of flexibility here, but we seldom use $d>4$ because then it gets *excessively* flexible.

- Prediction very straightforward:

$$ \hat{f}(x_0) = \hat{\beta}_0 + \hat{\beta}_1x_0 + \hat{\beta}_2x_0^2 +\cdots + \hat{\beta}_dx_0^d $$

## Polynomial Regression

![img](https://github.com/umbertomig/POLI175public/blob/main/img/polyreg1.png?raw=true)

## Polynomial Regression

In [ ]:
## Create the polynomials (with interaction terms!)
poly = PolynomialFeatures(degree = 4)
X = educ.reset_index()['income'].to_frame()
X = poly.fit_transform(X)
print(list(poly.get_feature_names_out(['income'])))

## Polynomial Regression

In [ ]:
## Fit a linear regression
reg = LinearRegression().fit(X, y)
print('The MSE for a quartic polynomial is: ' + str(np.mean((reg.predict(X) - y) ** 2)))

## Polynomial Regression

In [ ]:
# For the fully-specified model (lots of interactions...)
poly = PolynomialFeatures(degree = 4)
X = educ.reset_index()[['income', 'urban', 'young']]
X = poly.fit_transform(X)
print(list(poly.get_feature_names_out(['income', 'urban', 'young'])))
## Fit a linear regression
reg = LinearRegression().fit(X, y)
print('\n\nThe MSE for a quartic polynomial on income, urban, and \nyoung + interactions, is: ' + str(np.mean((reg.predict(X) - y) ** 2)))

## Polynomial Regression

In [ ]:
## With cross-validation now

# Only income
X = educ.reset_index()['income'].to_frame()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 12345)
X_train = poly.fit_transform(X_train)
X_test = poly.fit_transform(X_test)
reg = LinearRegression().fit(X_train, y_train)
print('The CV-MSE for a quartic polynomial on income is: ' + str(np.mean((reg.predict(X_test) - y_test) ** 2)))

## Polynomial Regression

In [ ]:
## With cross-validation now

# For the fully-specified model (lots of interactions...)
X = educ.reset_index()[['income', 'urban', 'young']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 12345)
X_train = poly.fit_transform(X_train)
X_test = poly.fit_transform(X_test)
reg = LinearRegression().fit(X_train, y_train)
print('The CV-MSE for a quartic polynomial on income, urban, and \nyoung + interactions, is: ' + str(np.mean((reg.predict(X_test) - y_test) ** 2)))

## Piecewise-constant Regression

- Here, we would be breaks in the predictor, making it an ordered categorical variable instead of a continuous variable.

- Let a variable $X$, the indicator function $I(.)$, and a set of cutpoints $\{c_1, c_2, \cdots, c_k\}$. The stepped variable $X$, $C_j(X)$, becomes:

$$
\begin{align}
C_{0}(X) &= I(X < c_1) \\
C_{1}(X) &= I(c_1 \leq X < c_2) \\
C_{2}(X) &= I(c_2 \leq X < c_3) \\
&\vdots\\
C_{K-1}(X) &= I(c_{K-1} \leq X < c_K) \\
C_{K}(X) &= I(X \geq c_K)
\end{align}
$$

## Piecewise-constant Regression

- Example use: when age is defined in 5-years bins.

- Detail: $C_0(x_i) + C_1(x_i) + \cdots + C_K(x_i) = 1$. This would make adding all pieces impossible:
    + Perfect Collinearity.

- The regression model becomes:

$$ y_i = \beta_0 + \beta_1C_1(x_i) + \beta_2C_2(x_i) +\cdots + \beta_KC_K(x_i) + \varepsilon_i $$

## Piecewise-constant Regression

![img](https://github.com/umbertomig/POLI175public/blob/main/img/pcreg1.png?raw=true)

## Piecewise-constant Regression

In [ ]:
## Piecewise-constant Regression
X = educ.reset_index()['income'].to_frame()
X = pd.cut(X.income, bins = [0, 2500, 3000, 3200, 3500, 4000, 5000])
X = pd.get_dummies(X, drop_first = True)
reg = LinearRegression().fit(X, y)
print('The MSE for this piecewise constant regression is: ' + str(np.mean((reg.predict(X) - y) ** 2)))

## Polynomial Regression

In [ ]:
## With cross-validation now
X = educ.reset_index()['income'].to_frame()
X = pd.cut(X.income, bins = [0, 2500, 3000, 3200, 3500, 4000, 5000])
X = pd.get_dummies(X, drop_first = True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 12345)
X_train = poly.fit_transform(X_train)
X_test = poly.fit_transform(X_test)
reg = LinearRegression().fit(X_train, y_train)
print('The CV-MSE for this piecewise constant regression is: ' + str(np.mean((reg.predict(X_test) - y_test) ** 2)))

# Questions?

# See you in the next class!